In [1]:
import numpy as np
from operator import itemgetter
import pandas as pd
import pathlib  
import html 


In [2]:
def add_quotes(val):
    if isinstance(val, str) and not ('\"' in val):
        return f'\"{val}\"'
    return val


def replace_quotes(val):
    if isinstance(val, str):
        return val.replace('\"', '\'').replace('\"', '\'')
    return val


def replace_para(val):
    if isinstance(val, str):
        return val.replace('(', '\t').replace(')', '\t')
    return val

In [3]:
raw_paths = [  pathlib.Path().cwd()/"data" /"soc-pokec-profiles.txt", 
               pathlib.Path().cwd()/"data"/"soc-pokec-relationships.txt"]

node_fields = [
            "public",
            "completion_percentage",
            "gender",
            "region",
            "last_login",
            "registration",
            "AGE",
            "body",
            "I_am_working_in_field",
            "spoken_languages",
            "hobbies",
            "I_most_enjoy_good_food",
            "pets",
            "body_type",
            "my_eyesight",
            "eye_color",
            "hair_color",
            "hair_type",
            "completed_level_of_education",
            "favourite_color",
            "relation_to_smoking",
            "relation_to_alcohol",
            "sign_in_zodiac",
            "on_pokec_i_am_looking_for",
            "love_is_for_me",
            "relation_to_casual_sex",
            "my_partner_should_be",
            "marital_status",
            "children",
            "relation_to_children",
            "I_like_movies",
            "I_like_watching_movie",
            "I_like_music",
            "I_mostly_like_listening_to_music",
            "the_idea_of_good_evening",
            "I_like_specialties_from_kitchen",
            "fun",
            "I_am_going_to_concerts",
            "my_active_sports",
            "my_passive_sports",
            "profession",
            "I_like_books",
            "life_style",
            "music",
            "cars",
            "politics",
            "relationships",
            "art_culture",
            "hobbies_interests",
            "science_technologies",
            "computers_internet",
            "education",
            "sport",
            "movies",
            "travelling",
            "health",
            "companies_brands",
            "more",
            ""
        ]
dfn = pd.read_csv(raw_paths[0], sep = "\t", names = node_fields, nrows = None)
dfe = pd.read_csv(raw_paths[1], sep = "\t", names = ["source", "target"], nrows = None)


# transform nodes

dfn =dfn.map(lambda x: html.escape(x) if isinstance(x, str) else x)
dfn = dfn.map(add_quotes)
# dfn["last_login"] = dfn["last_login"].apply(lambda row: f"""dateTime({row})""" )
# dfn["registration"] = dfn["registration"].apply(lambda row: f"""dateTime({row})""") 
# transform edges
dfe = dfe.astype({'source': 'str', 'target': 'str'})
node_frame = dfn
edge_frame = dfe

# save as hdf
# store["dfn"] = dfn
# store["dfe"] = dfe
# store.close()
len(dfn) * len(node_fields)

96335377

In [4]:
len(dfn)

1632803

In [40]:
# transform nodes

node_strings = dfn.apply(
    lambda row: f"N{row.name}:people "+' '.join([f"{col}:{row[col]}" for col in dfn.columns if pd.notna(row[col])]),
    axis=1
)

edge_strings = dfe.apply(
    lambda row: f"N{row["source"]} -> N{row["target"]}: follow",
    axis=1
)


In [43]:
with open(pathlib.Path().cwd()/"data" /"soc-pokec.qm",'w') as f:
    for row in node_strings:
        f.write(row + "\n")
    for row in edge_strings:
        f.write(row + "\n")

In [1]:
len(dfe) + len(dfn)

NameError: name 'dfe' is not defined